Wholesale data:
https://archive.ics.uci.edu/ml/datasets/Wholesale+customers
The dataset given show how much annual spending is done on various products, at three regions (Lisbon, Oporto, Others), and where they will be used (HoReCa (Hotel, Restaurant, Cafe) or Retail).
This dataset also refers to 'clients of a wholesale distributor', which means that this would be one organisation selling 440 products to various customers.
It is possible that one customer may NOT buy multiple products, such as buying only Fresh and Milk products.

This Jupyter Notebook will predict whether a customer will use products for either HoReCa use or Retail use.
This prediction will be done using the 'Random Forest' method.

In [ ]:
import pandas as pd
# This Notebook will predict, given a selection of products, whether the customer will be using the products for HoReCa usage (Hotel, Restaurant, Cafe) or Retail usage (Reselling).
#
# Will be using the pandas data analysis library

In [ ]:
df = pd.read_csv('Wholesale customers data.csv')
df.head()

# Channel - Refers to where the products will be used.
# 1 = HoReCa (Hotel, Restaurant, Cafe)
# 2 = Retail
#
# Region - Refers to where the customers are based. This will not be used as we are interested in how much an average customer would spend in each of the products in either HoReCa or Retail.
#
# Fresh, Milk, Grocery, Frozen, Detergents Paper, Delicassen
# These are our products for what a customer orders, and the annual spending on the respective products in M.U. (Monetary Unit)

In [ ]:
df.shape

In [ ]:
df.isnull().values.any()
# Checking if any of the values are missing

In [ ]:
df = df[["Channel", "Fresh", "Milk", "Grocery", "Frozen", "Detergents_Paper", "Delicassen"]]
df.head()
# Removing Region as it is no longer needed as part of this data analysis

In [ ]:
df_temp = df.values.flatten()
any(df_temp < 0)
# Checks if there are any values under 0
# Cleaning Data

In [ ]:
sum(n < 0 for n in df.values.flatten())
# Checks if there are any values under 0, although a check has already been done, this can be used to sum the amount of times the 'n' condition appears in the dataset.
# Dataset must be flattened for this to work.
# Cleaning Data

In [ ]:
df.loc[df['Channel'] > 1, 'Channel'] = 0
df.head()
# As we now need a binary dataset, we must now set all occurences of '2' in the Channel column to be 0's.
#
# This now means that channel:
# 0 = Retail
# 1 = HoReCa

In [ ]:
from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y = train_test_split(df.drop('Channel', axis=1), df['Channel'], test_size = 0.2, random_state=11)
# As the dataset is now cleaned, we will now split the data into 80-20 training-test dataset.
# X will hold now hold the spending done by a given customer.
# Y will determine the channel (0 or 1), which corresponds to Retail or HoReCa use.

In [ ]:
train_x.shape

In [ ]:
test_x.shape

In [ ]:
train_x.head()

In [ ]:
test_x.head()

In [ ]:
train_y.shape

In [ ]:
test_y.shape

In [ ]:
train_y.head()

In [ ]:
test_y.head()

In [ ]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(random_state=21)
model.fit(train_x, train_y)

# Now training the data using our split dataset from before, using SKLearn's Random Forest method.

In [ ]:
predicted = model.predict(test_x)
model.score(test_x, test_y)
# This model has 93% mean accuracy.
# However, mean accuracy is not the best way to check the classification model's accuracy.

In [ ]:
from sklearn.metrics import roc_auc_score
probabilities = model.predict_proba(test_x)
# ROC AUC checks are an overall best way for binary classification models
# ROC AUC is how often the model will make a correct prediction regardless to the outcome.

In [ ]:
roc_auc_score(test_y, probabilities[:, 1])
# The percentage increases a little.
# However, there is an imbalance in the dataset, with 142 Retails and 298 HoReCa occurences.
# There are nearly twice as many HoReCa occurences than Retail occurences.
# This means that the model trains on twice as much HoReCa occurences, but the prediction is whether or not a given prediction is Retail or not.

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(test_y, predicted)
# The small testing size might have skewed the following predictions heavily, considering the dataset is only 440.
# However, this is dataset given from a SINGLE company with nearly 440 customers.
# A more accuracte prediction would need more companies to share data about their customers
# As for now, conclusions that can be made for this company can only apply to this company.

In [ ]:
from sklearn.metrics import precision_score

train_predictions = model.predict(train_x)
precision_score(train_y, train_predictions)

In [ ]:
from sklearn.metrics import recall_score

recall_score(train_y, train_predictions)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

sns.set()
# Enables Jupyter to render Matplotlib outputs
# Must appear before any references to Matplotlib itself
# Final statement configures Seaborn to enhance output from Matplotlib

In [ ]:
from sklearn.metrics import roc_curve

fpr, tpr, _ = roc_curve(test_y, probabilities[:, 1])
plt.plot(fpr, tpr)
plt.plot([0, 1], [0, 1], color='grey', lw=1, linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')

# Again shows the heavily skewed data from the small data size.

In [ ]:
def predict_use(Fresh, Milk, Grocery, Frozen, Detergents, Delicassen):
    
    # Checks for negative values, as this can not happen.
    # Returns -1 to show an error
    if Fresh < 0 or Milk < 0 or Grocery < 0 or Frozen < 0 or Detergents < 0 or Delicassen < 0:
        return -1
    else:
        input = [{'Fresh': Fresh,
                 'Milk' : Milk,
                 'Grocery' : Grocery,
                 'Frozen' : Frozen,
                 'Detergents_Paper' : Detergents,
                 'Delicassen' : Delicassen}]
        return model.predict_proba(pd.DataFrame(input))[0][0]

In [ ]:
predict_use(12000, 10000, 7500, 200, 2500, 1400)
# Testing normal usage

In [ ]:
predict_use(12000, -2000, 7500, 200, 2500, 1400)
# Testing a negative parameter

In [ ]:
import numpy as np

plt.rcdefaults()
fig, ax = plt.subplots()

labels = ("Low F, High D", "High F, Low D", "Random", "Known HoReCa", "Known Retail")
values = (predict_use(10000, 10000, 7500, 200, 5000, 1234), # Low Frozen, High Detergents
         predict_use(5000, 5000, 200, 7500, 1234, 5678),  # High Frozen, Low Detergents
         predict_use(12345, 67891, 2345, 6789, 1234, 5678),  # Random Values
         predict_use(13000, 1200, 4200, 6400, 500, 1800),  # Known HoReCa from Dataset
         predict_use(6400, 8800, 7600, 2400, 3500, 7800))  # Known Retail from Dataset

y_labels = np.arange(len(labels))

#plt.bar(alabels, values, align='center', alpha=0.5)
#plt.xticks(alabels, labels)
#plt.ylabel('Probability of Retail or HoReCa')
#plt.ylim((0.0, 1.0))

ax.barh(y_labels, values, align='center')
ax.set_yticks(y_labels)
ax.set_yticklabels(labels)
ax.invert_yaxis()  # labels read top-to-bottom
ax.set_xlabel('Probability of being used for Retail')
ax.set_title('Wholesale Predictions')

plt.show()

# This graph shows a bunch of predictions made with a bunch of different conditions